In [1]:
import PySimpleGUI as sg
import os.path
import requests
import cv2
import sqlite3
import os
import shutil

In [2]:
class Food_ai():
    headers = {'Prediction-Key': '93cf8213e5cb4a1e8432e0fece2449db',
           'Content-Type': 'application/octet-stream' }
    food_url = 'https://eastus.api.cognitive.microsoft.com/customvision/v3.0/Prediction/a36ad249-5c52-420f-808f-ee205f9e3ae6/classify/iterations/Iteration2/image'
    category = ''
    def __init__(self,image_path):
        self.image_data = open(image_path, 'rb').read()
    
    def print_type_and_prob(self):
        response = requests.post(url=Food_ai.food_url, headers=Food_ai.headers, data=self.image_data)
        Food_ai.category = eval(response.text)['predictions'][0]['tagName']
        return "Probability: " + str(eval(response.text)['predictions'][0]['probability']) + "\n" + "Type: " + eval(response.text)['predictions'][0]['tagName']
   # def return_category(self):
   #     return self.catego

In [3]:
class Take_picture():
    def __init__(self):
        cap = cv2.VideoCapture(0)#创建一个 VideoCapture 对象

        flag = 1 #设置一个标志，用来输出视频信息
        num = 1 #递增，用来保存文件名
        while(cap.isOpened()):#循环读取每一帧
            ret_flag, Vshow = cap.read() #返回两个参数，第一个是bool是否正常打开，第二个是照片数组，如果只设置一个则变成一个tumple包含bool和图片
            cv2.imshow("Take Food Picture",Vshow)  #窗口显示，显示名为 Capture_Test
            k = cv2.waitKey(1) & 0xFF #每帧数据延时 1ms，延时不能为 0，否则读取的结果会是静态帧
            if k == ord('s'):  #若检测到按键 ‘s’，打印字符串
                cv2.imwrite("C:/Users/Junhao's Alienware/food_recognition/food_picture/"+ str(num) + ".png", Vshow)
                print(cap.get(3)); #得到长宽
                print(cap.get(4));
                print("success to save"+str(num)+".png")
                print("-------------------------")
                num += 1
            elif k == ord('q'): #若检测到按键 ‘q’，退出
                break
        cap.release() #释放摄像头
        cv2.destroyAllWindows()#删除建立的全部窗口

In [4]:


# 连接到SQLite数据库
# 数据库文件是buy_value.db
# 如果文件不存在，会自动在当前目录创建:
conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
# 创建一个Cursor:
cursor = conn.cursor()
# 执行一条SQL语句，创建user表:
cursor.execute('create table customer (id varchar(20), value float)')
# 提交事务:
conn.commit()
# 关闭Connection:
conn.close()

sg.theme('Dark')   # 设置当前主题
file_list_column = [
    [
        sg.Text("功能按钮"),
#         sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
#         sg.FolderBrowse('浏览'),
        sg.Button("拍照(s键为拍照，q键为退出)", key="Take_photo"),
#         '拍照（s是拍照，q是退出）', key=
        sg.Button("结算", key="Check_out"),
        
#         '结算', key=
    ],
    [   sg.Button("重置", key="Reset")
    ],
    [   sg.Text("以下为食品照片清单：")
    ],

    [
        sg.Listbox(
            values=[], enable_events=True, size=(40, 20), key="-FILE LIST-"
        )
    ],
]
image_viewer_column = [
#     [sg.Text("左侧为所有拍过的食品图片清单")],
    [sg.Text(size=(40, 1), key="-TOUT-")],
    [sg.Listbox(values=[], size=(40, 20), key="-POUT-")],
#     [sg.Text(size=(20, 1), key="-TOUT-")],
    [sg.Text(size=(40, 1), key="-SOUT-")],
#     [sg.Image(key="-IMAGE-")],
]
layout = [
    [
        sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(image_viewer_column),
    ]
]
window = sg.Window("照片识别支付系统", layout)
while True:
    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
    if event == "Reset":
        try:
            window["-TOUT-"].update('')
            window["-POUT-"].update('')
            window["-SOUT-"].update('')
            window["-FILE LIST-"].update('')
            # 连接到SQLite数据库
            # 数据库文件是buy_value.db
            # 如果文件不存在，会自动在当前目录创建:
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            # 创建一个Cursor:
            cursor = conn.cursor()
            # 执行一条SQL语句，创建user表:
            cursor.execute('create table customer (id varchar(20), value float)')
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
        except:
            pass
    if event == "Take_photo":
        tke_picture = Take_picture()
    if event == "Check_out":
        folder = "C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture"
        try:
            file_list = os.listdir(folder)
#             print(file_list)
        except:
            file_list = []
        try:
            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".png", ".gif"))
            ]
    #         print(fnames)
            window["-FILE LIST-"].update(fnames)
            print('update success')
            list1 = []
            for pic_name in fnames:
                print(pic_name)
    #             filename = os.path.join(
    #                 values["-FOLDER-"], pic_name
    #             )
                filename = "C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture" + '\\' + pic_name
    #            print(filename)
                food_rec = Food_ai(filename)
                food_rec.print_type_and_prob()
                print(food_rec.category)
                list1.append(food_rec.category)
                print(list1)
            #将扫描得出的产品类遍历输入buy_value数据库中
            for category_name in list1:           
                # 如果文件不存在，会自动在当前目录创建:
                conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/name_value.db")
                # 创建一个Cursor:
                cursor = conn.cursor()
                cursor.execute('select * from user where id=?', (category_name,))
                # 获得查询结果集:
                value = cursor.fetchall()
                print(value)
                # 提交事务:
                conn.commit()
                # 关闭Connection:
                conn.close()
                conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
                cursor = conn.cursor()
                cursor.execute("insert into customer (id, value) values (?, ?)", (value[0][0], float(value[0][1])))
                # 提交事务:
                conn.commit()
                # 关闭Connection:
                conn.close()
            #把数据输出到GUI
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            cursor = conn.cursor()
            # 执行查询语句:
            cursor.execute('select * from customer')
            # 获得查询结果集:
            values = cursor.fetchall()
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
            window["-TOUT-"].update('扫描产品及价格清单：')
            window["-POUT-"].update(values)
            conn = sqlite3.connect("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            cursor = conn.cursor()
            # 执行查询语句:
            cursor.execute('select sum(value) from customer')
            # 获得查询结果集:
            sum_value = cursor.fetchall()
            # 提交事务:
            conn.commit()
            # 关闭Connection:
            conn.close()
            window["-SOUT-"].update('总金额合计（元）：' + str(round(sum_value[0][0],2)))
            os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
            shutil.rmtree("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
            os.mkdir("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")
    #     elif event == "-FILE LIST-":  
    #         try:
    #             filename = os.path.join(
    #                 values["-FOLDER-"], values["-FILE LIST-"][0]
    #             )
    # #             filename = values["-FILE LIST-"][0]
    #             print(filename)
    #             print(type(filename))
    #             window["-TOUT-"].update(filename)
    #             food_rec = Food_ai(filename)
    #             window["-POUT-"].update(food_rec.print_type_and_prob())
    #             window["-IMAGE-"].update(filename=filename)
    #             image_data = open(filename, 'rb').read()

    #         except:
    #             pass
        except:
            pass
try:
    os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")
except:
    pass
window.close()

640.0
480.0
success to save1.png
-------------------------
640.0
480.0
success to save2.png
-------------------------
640.0
480.0
success to save3.png
-------------------------
640.0
480.0
success to save4.png
-------------------------
update success
1.png
Chilled_noodles
['Chilled_noodles']
2.png
Chilled_noodles
['Chilled_noodles', 'Chilled_noodles']
3.png
Chilled_noodles
['Chilled_noodles', 'Chilled_noodles', 'Chilled_noodles']
4.png
Chilled_noodles
['Chilled_noodles', 'Chilled_noodles', 'Chilled_noodles', 'Chilled_noodles']
[('Chilled_noodles', 49.0)]
[('Chilled_noodles', 49.0)]
[('Chilled_noodles', 49.0)]
[('Chilled_noodles', 49.0)]
640.0
480.0
success to save1.png
-------------------------
640.0
480.0
success to save2.png
-------------------------
640.0
480.0
success to save3.png
-------------------------
update success
1.png
Doughnut
['Doughnut']
2.png
Egg_pudding
['Doughnut', 'Egg_pudding']
3.png
Doughnut
['Doughnut', 'Egg_pudding', 'Doughnut']
[('Doughnut', 13.4)]
[('Egg_puddin

In [86]:
image_path = "C:/Users/Junhao's Alienware/Desktop\下载.png"

In [87]:
image_data = open(image_path, 'rb').read()

In [9]:
food_rec = Food_ai("C:/Users/Junhao's Alienware/Desktop/下载.png")

In [89]:
print(food_rec.print_type_and_prob())

{'probability': 0.99999666, 'tagId': '4d5086e5-535c-4f30-8e9f-eb2e90d2b37c', 'tagName': '冰激凌'}


In [3]:
os.remove("C:\\Users\\Junhao's Alienware\\food_recognition\\Database/buy_value.db")

In [10]:
shutil.rmtree("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")

In [11]:
os.mkdir("C:\\Users\\Junhao's Alienware\\food_recognition\\Food_picture")